<h1 align="center">Modelling Uncertainity using Mixture Density Networks</h1>

<h4 align="center">Jaison Saji Chacko</h4>

<h2 align="center">Categorical vs Continuos models</h2>
<img src="images/categorical_vs_continuos.png"/>

<h2 align="center">Normal Gaussian Distribution</h2>

<img align="left" src="images/normal_distribution.png"/>
<h4 align="left">"Standard" probability distribution has two parameter:</h4>


       


<h2 align="center">Normal Gaussian Distribution</h2>


<img align="left" src="images/normal_distribution.png"/>
<h3 aligh="left">"Standard" probability distribution has two parameter:</h3>
</br>
<ul align="right">
    <li>mean(μ)</li>
    <li>standard deviation (σ)</li>
</ul>
       


<h2 align="center">Normal Gaussian Distribution</h2>


<img align="left" src="images/unimodal_normal_fit.png"/>
<h3>"Standard" probability distribution has two parameter:</h3>

<ul align="right">
    <li>mean(μ)</li>
    <li>standard deviation (σ)</li>
</ul>
       

<b align="right">Probability Density Function</b>

<img src="images/pdf.png"/>
       


<h2 align="center">Problem: Normal Distribution might not fit data</h2>

<h3>What if the data is not "normal"?</h3>
<img src="images/multimodal_normal_fit.png"/>




<h2 align="center">Solution: Mixture of gaussian models</h2>

<img align="left" src="images/mixture_model_fit.png"/>


<h2 align="center">Solution: Mixture of gaussian models</h2>

<img align="left" src="images/mixture_model_fit.png"/>

<h3> Mixture models have 3 parameters </h3>

     

<h2 align="center">Solution: Mixture of gaussian models</h2>

<img align="left" src="images/mixture_model_fit.png"/>

<h3> Mixture models have 3 parameters </h3>

<ul align="right">
    <li>mean(μ): location of the component</li>
    <li>standard deviation (σ): width of the component &</li>
    <li>Weight (π): height of the component</li>
</ul>
     

<h2 align="center">Solution: Mixture of gaussian models</h2>

<img align="left" src="images/mixture_model_fit.png"/>

<h3> Mixture models have 3 parameters </h3>

<ul align="right">
    <li>mean(μ): location of the component</li>
    <li>standard deviation (σ): width of the component &</li>
    <li>Weight (π): height of the component</li>
</ul>
       

<b align="right">Probability Density Function</b>

<img src="images/mdn_pdf.png"/>



<h2 align="center"> Mixture Density Networks </h2>


<img src="images/mdn_unimodal.png"/>

Modelling complicated real-valued data that might not be "normal".

<ul>
    <li>Naive approach: single neuron output with no squashing function.</li>
    <li>training function =  MSE (mean squared error)</li>
</ul>

<h2 align="center"> Mixture Density Networks </h2>


<img src="images/mdn_unimodal.png"/>

Modelling complicated real-valued data that might not be "normal".

<ul>
    <li>Naive approach: single neuron output with no squashing function.</li>
    <li>training function =  MSE (mean squared error)</li>
    <li><b>This is same as fitting the data to a single normal distribution</b></li>
</ul>

<h2 align="center"> Mixture Density Networks </h2>


<img src="images/mdn_multimodal.png"/>


<h2 align="center"> MDN-RNNS </h2>

<img src="images/robot_path.png"/>

<h2 align="center"> MDN-RNNS </h2>

<img src="images/mdn_rnn.png"/>

<h2 align="center">Use cases</h2>

<h3>Sketch RNN</h3>

<img src="images/sketch_rnn.png"/>

<a href="https://magenta.tensorflow.org/assets/sketch_rnn_demo/index.html">DEMO</a>




<h2 align="center">Use cases</h2>
<h3>World Model</h3>

<img src="images/mdn-rnn-viz.png"/>

<a href="https://worldmodels.github.io/">DEMO</a>

<h2 align="center">Use cases</h2>
<h3>Dancenet</h3>
<i>http://github.com/jsn5/dancenet</i>
<h4>Dance Generator AI using MDN-RNN and VAE</h4>
<img align="left" src="images/dancenet_demo.gif"/>
<img align="left" src="images/dancenet_demo2.gif"/>


<h2 align="center">Use cases</h2>
<h3>Dancenet</h3>

<img src="images/dancenet_vae.png"/>